# 🎮 Keep the Land! ML-Agents Training on Google Colab

## Paper.io 스타일 영역 확보 게임 프로젝트

**프로젝트 개요:** Unity ML-Agents를 활용한 Keep the Land! 게임에서 4개의 AI 에이전트가 PPO 강화학습을 통해 최적의 전략을 학습하는 프로젝트입니다.

---

### 📋 이 노트북의 구성
1. **환경 설정** - Python 버전 확인 및 ML-Agents 설치
2. **프로젝트 준비** - GitHub에서 Keep the Land! 프로젝트 클론
3. **Unity 빌드 업로드** - 헤드리스 빌드 파일 준비
4. **학습 설정** - 하이퍼파라미터 구성
5. **학습 실행** - TensorBoard와 함께 학습 진행
6. **모델 다운로드** - 학습된 .onnx 파일 저장

### ⚡ 시작하기 전에
- **GPU 런타임** 설정: 런타임 → 런타임 유형 변경 → GPU (T4) 선택
- **Unity 헤드리스 빌드** 파일이 필요합니다 (Linux 플랫폼)
- 예상 학습 시간: **30분 ~ 2시간** (설정에 따라)

## 🔧 Step 1: 환경 설정 및 Python 버전 확인

In [ ]:
import sys
import os

print("🐍 Python 버전 확인:")
print(f"Version: {sys.version}")
print(f"Platform: {sys.platform}")

if sys.version_info[0] < 3:
    raise Exception("❌ ERROR: ML-Agents는 Python 3가 필요합니다!")
else:
    print("✅ Python 3 확인됨!")

# GPU 확인
print("\n🖥️ GPU 상태 확인:")
!nvidia-smi -L

## 📦 Step 2: Python 버전 호환성 확인 및 ML-Agents 설치

### ⚠️ 중요: Python 버전 호환성
- **ML-Agents 0.30.0**: Python 3.8 ~ 3.10 호환
- **Colab 기본 Python 3.11+**: ML-Agents와 호환 문제 발생
- **자동 해결**: Python 3.11+ 감지 시 3.10.12로 컴파일 설치 (5분 소요)

### 🔄 처리 방식
1. **Python 3.10**: ML-Agents 0.30.0 직접 설치
2. **Python 3.8-3.9**: ML-Agents 0.28.0 안정 버전 설치  
3. **Python 3.11+**: 컴파일 방식으로 3.10.12 다운그레이드 후 재시작

In [ ]:
print("📥 ML-Agents 설치 중...")

# 현재 Python 버전 확인
import sys
python_version = f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}"
print(f"🐍 현재 Python 버전: {python_version}")

# Python 3.11+ 감지 시 3.10.12로 안전하게 다운그레이드
if sys.version_info >= (3, 11):
    print("⚠️ Python 3.11+ 감지! ML-Agents 호환을 위해 Python 3.10.12로 다운그레이드 필요!")
    print("🔄 Python 3.10.12 컴파일 설치 중... (약 5분 소요)")
    
    # wooono.tistory.com/478 방법 적용 (더 안전하고 안정적)
    # Python 소스 다운로드 및 컴파일 설치
    !wget -q https://www.python.org/ftp/python/3.10.12/Python-3.10.12.tgz
    !tar xzf Python-3.10.12.tgz
    
    # 컴파일 설치 (최적화 포함)
    import os
    original_dir = os.getcwd()
    os.chdir('/content/Python-3.10.12')
    
    !./configure --enable-optimizations --prefix=/usr/local
    !make -j$(nproc)
    !sudo make install
    
    # 원래 디렉토리로 복귀
    os.chdir(original_dir)
    
    # 새 Python으로 심볼릭 링크 생성
    !sudo ln -sf /usr/local/bin/python3.10 /usr/bin/python3
    !sudo ln -sf /usr/local/bin/python3.10 /usr/bin/python
    !sudo ln -sf /usr/local/bin/pip3.10 /usr/bin/pip
    
    # pip 업그레이드
    !/usr/local/bin/python3.10 -m pip install --upgrade pip
    
    print("✅ Python 3.10.12 컴파일 설치 완료!")
    print("🔄 런타임 재시작이 필요합니다!")
    print("")
    print("📝 다음 단계:")
    print("1. ⚠️  런타임 → 런타임 재시작 클릭")
    print("2. 🔄 재시작 후 이 Step 2 셀을 다시 실행")
    print("3. ✅ Python 3.10으로 ML-Agents 정상 설치")
    
    # 런타임 강제 재시작
    import os
    os.kill(os.getpid(), 9)
    
elif sys.version_info >= (3, 10) and sys.version_info < (3, 11):
    print("✅ Python 3.10 감지! ML-Agents 호환 버전입니다.")
    print("📦 ML-Agents 0.30.0 설치 중...")
    
    # 캐시 정리 및 pip 업그레이드
    !pip cache purge
    !pip install --upgrade pip setuptools wheel -q
    
    # ML-Agents 최적 버전 설치 (더 안정적인 방법)
    !pip install protobuf==3.20.3 -q
    !pip install grpcio==1.48.2 -q
    !pip install mlagents==0.30.0 -q
    !pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 -q
    
    print("✅ ML-Agents 설치 완료!")
    
    # 설치 확인
    print("\n🔍 ML-Agents 설치 확인:")
    try:
        import mlagents
        print(f"✅ mlagents 모듈 가져오기 성공 (버전: {mlagents.__version__})")
        
        # 명령어 확인 (더 안전한 방법)
        try:
            !mlagents-learn --help | head -3
            print("✅ mlagents-learn 명령어 사용 가능")
        except:
            print("ℹ️ 직접 명령어 사용 불가, python -m 방식 사용")
        
    except Exception as e:
        print(f"⚠️ mlagents 가져오기 실패: {e}")
        
    # 대체 명령어 확인
    try:
        !python -m mlagents.trainers.learn --help | head -3
        print("✅ python -m mlagents.trainers.learn 사용 가능")
    except Exception as e2:
        print(f"❌ ML-Agents 설치 확인 실패: {e2}")
    
    # 최종 버전 확인
    print("\n📦 설치된 패키지 버전:")
    !pip show mlagents | grep Version
    !python --version
    
elif sys.version_info >= (3, 8):
    print("✅ Python 3.8-3.9 감지! 안정적인 ML-Agents 버전으로 설치...")
    
    # 캐시 정리 및 pip 업그레이드
    !pip cache purge
    !pip install --upgrade pip setuptools wheel -q
    
    # 안정적인 ML-Agents 버전 (Python 3.8-3.9 호환)
    !pip install protobuf==3.20.3 -q
    !pip install grpcio==1.48.2 -q
    !pip install mlagents==0.28.0 -q
    !pip install torch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1 -q
    
    print("✅ ML-Agents 0.28.0 설치 완료!")
    
    # 설치 확인
    print("\n🔍 ML-Agents 설치 확인:")
    try:
        import mlagents
        print(f"✅ mlagents 모듈 가져오기 성공 (버전: {mlagents.__version__})")
    except Exception as e:
        print(f"⚠️ mlagents 가져오기 실패: {e}")
    
    print("\n📦 설치된 패키지 버전:")
    !pip show mlagents | grep Version
    !python --version
    
else:
    print("❌ Python 버전이 너무 낮습니다! 3.8 이상이 필요합니다.")
    print(f"현재 버전: {python_version}")
    raise Exception("Python 3.8+ required for ML-Agents")

print("\n🎉 Step 2 완료! 다음 Step으로 진행하세요.")

## 📂 Step 3: Keep the Land! ML-Agents 프로젝트 준비

In [ ]:
import os

print("📂 Keep the Land! ML-Agents 프로젝트 준비...")

# 작업 디렉토리로 이동
os.chdir('/content')

# 프로젝트 폴더 생성
!mkdir -p Project-AI-Paper.io-ML-Agents
os.chdir('/content/Project-AI-Paper.io-ML-Agents')

print("✅ 작업 폴더 준비 완료!")
print("📝 참고: Unity 빌드 파일은 Colab 세션에 미리 업로드해주세요.")

print("\n📁 현재 작업 디렉토리:")
!pwd
!ls -la

## 🏗️ Step 4: Unity Linux 빌드 파일 확인

### ⚡ 빠른 방식: 세션에 미리 업로드된 파일 사용

**Colab 세션**에 미리 업로드해놓은 Unity Linux Server 빌드 파일을 확인합니다.

#### 📁 필요한 빌드 파일들:
- **PaperIO_JM_Linux.x86_64** (실행 파일)
- **PaperIO_JM_Linux_Data/** (폴더 전체 - 169개 파일 포함)
- **UnityPlayer.so**

#### 🔧 사전 준비사항:
1. **Juwon/250703/Linux** 브랜치에서 빌드 파일들을 가져옴
2. **Colab 세션 파일 탭**에서 빌드 파일들을 `/content/` 또는 적절한 위치에 업로드
3. 아래 코드로 파일 위치 확인 및 설정

#### 📍 빌드 파일 위치 확인:

In [ ]:
import os
import zipfile

print("🔍 Colab 세션에 업로드된 Unity 빌드 파일 찾기...")

# 가능한 빌드 파일 위치들 확인
possible_locations = [
    '/content/',
    '/content/build/',
    '/content/Project-AI-Paper.io-ML-Agents/',
]

build_found = False
build_location = None

for location in possible_locations:
    print(f"\n📂 {location} 확인 중...")
    if os.path.exists(location):
        files_in_location = os.listdir(location)
        print(f"   파일들: {files_in_location}")
        
        # PaperIO_JM_Linux.x86_64 파일 찾기
        for file in files_in_location:
            if file == 'PaperIO_JM_Linux.x86_64':
                build_location = location
                build_found = True
                print(f"   ✅ 빌드 실행 파일 발견: {location}{file}")
                break
            elif file.endswith('.zip') and 'Linux' in file:
                # ZIP 파일인 경우 압축 해제
                print(f"   📦 ZIP 파일 발견: {file}")
                print(f"   압축 해제 중...")
                
                extract_path = os.path.join(location, 'build')
                os.makedirs(extract_path, exist_ok=True)
                
                with zipfile.ZipFile(os.path.join(location, file), 'r') as zip_ref:
                    zip_ref.extractall(extract_path)
                
                # 압축 해제 후 실행 파일 확인
                if os.path.exists(os.path.join(extract_path, 'PaperIO_JM_Linux.x86_64')):
                    build_location = extract_path + '/'
                    build_found = True
                    print(f"   ✅ 압축 해제 완료: {build_location}")
                    break
        
        if build_found:
            break

if build_found:
    print(f"\n🎉 Unity 빌드 파일 발견!")
    print(f"📍 빌드 위치: {build_location}")
    
    # 실행 권한 부여
    print("\n🔐 실행 권한 설정...")
    !find {build_location} -name "*.x86_64" -exec chmod +x {{}} \;
    !find {build_location} -name "*.so" -exec chmod +x {{}} \;
    
    # 빌드 파일 구조 확인
    print(f"\n📁 빌드 파일 구조:")
    !ls -la {build_location}
    
    # Data 폴더 확인
    data_folder = os.path.join(build_location, 'PaperIO_JM_Linux_Data')
    if os.path.exists(data_folder):
        print(f"\n📊 Data 폴더 내용 (일부):")
        !ls -la {data_folder} | head -10
        
        # Managed 폴더 확인
        managed_folder = os.path.join(data_folder, 'Managed')
        if os.path.exists(managed_folder):
            print(f"\n🔧 Managed 어셈블리 확인:")
            !ls -la {managed_folder} | grep -E "(Unity|ML|netstandard)" | head -5
    
    # 전역 변수로 빌드 경로 저장
    BUILD_PATH = os.path.join(build_location, 'PaperIO_JM_Linux.x86_64')
    print(f"\n✅ 빌드 파일 준비 완료!")
    print(f"🎮 실행 파일 경로: {BUILD_PATH}")
    
else:
    print("\n❌ Unity 빌드 파일을 찾을 수 없습니다!")
    print("\n📝 해결 방법:")
    print("1. Colab 세션 파일 탭에서 PaperIO_JM_Linux.x86_64 업로드")
    print("2. 또는 빌드 파일들을 ZIP으로 압축해서 업로드")
    print("3. /content/ 폴더에 업로드하는 것을 권장")
    
    print(f"\n🔍 현재 /content/ 폴더 내용:")
    !ls -la /content/

## ⚙️ Step 5: 학습 설정 (하이퍼파라미터)

In [ ]:
# 환경 설정
# Step 4에서 설정된 BUILD_PATH 사용 (없으면 기본값)
try:
    ENV_NAME = BUILD_PATH
    print(f"✅ Step 4에서 찾은 빌드 파일 사용: {ENV_NAME}")
except NameError:
    # BUILD_PATH가 없는 경우 기본 경로들 시도
    possible_paths = [
        "./build/PaperIO_JM_Linux.x86_64",
        "/content/PaperIO_JM_Linux.x86_64",
        "/content/build/PaperIO_JM_Linux.x86_64"
    ]
    
    ENV_NAME = None
    for path in possible_paths:
        if os.path.exists(path):
            ENV_NAME = path
            break
    
    if ENV_NAME is None:
        ENV_NAME = "./build/PaperIO_JM_Linux.x86_64"  # 기본값
        print("⚠️ 빌드 파일을 찾을 수 없습니다. 기본 경로를 사용합니다.")
    else:
        print(f"✅ 빌드 파일 발견: {ENV_NAME}")

RUN_ID = "paperio_colab_run_001"  # 학습 실행 ID
MAX_STEPS = 5000000  # 최대 학습 스텝 (500만)

print(f"\n🎮 환경 이름: {ENV_NAME}")
print(f"🏷️ 실행 ID: {RUN_ID}")
print(f"📈 최대 스텝: {MAX_STEPS:,}")

# 빌드 파일 존재 확인
if os.path.exists(ENV_NAME):
    file_size = os.path.getsize(ENV_NAME) / 1024  # KB
    print(f"📊 빌드 파일 크기: {file_size:.1f}KB")
    if file_size < 100:
        print("⚠️ 경고: 빌드 파일이 비정상적으로 작습니다!")
    else:
        print("✅ 빌드 파일 크기 정상")
else:
    print("❌ 경고: 빌드 파일이 존재하지 않습니다!")

# Paper.io 전용 학습 설정 파일 생성
config_content = f"""behaviors:
  MyAgent:
    trainer_type: ppo
    hyperparameters:
      batch_size: 2048
      buffer_size: 20480
      learning_rate: 3.0e-4
      beta: 5.0e-3
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: linear
    network_settings:
      normalize: false
      hidden_units: 512
      num_layers: 3
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.99
        strength: 1.0
    max_steps: {MAX_STEPS}
    time_horizon: 1000
    summary_freq: 10000
    checkpoint_interval: 50000
    keep_checkpoints: 10
"""

# 설정 파일 저장
with open('paperio_config.yaml', 'w') as f:
    f.write(config_content)

print("\n✅ Paper.io 전용 설정 파일 생성 완료!")
print("\n📄 설정 내용:")
!cat paperio_config.yaml

## 📊 Step 6: TensorBoard 설정

In [ ]:
# TensorBoard 확장 로드
%load_ext tensorboard

# TensorBoard 시작 (백그라운드에서 실행)
%tensorboard --logdir results --port 6006

print("📊 TensorBoard가 시작되었습니다!")
print("학습이 시작되면 위 창에서 실시간 진행 상황을 확인할 수 있습니다.")

## 🚀 Step 7: Keep the Land! AI 학습 시작!

### 🎯 학습 목표
- **영역 확장**: AI가 효율적으로 영역을 넓히는 전략 학습
- **생존 전략**: 상대방의 공격을 피하고 안전하게 복귀
- **공격 전략**: 상대방의 궤적을 끊어 점수 획득
- **위험 회피**: 벽 충돌이나 자살 방지

### ⏱️ 예상 소요 시간
- **초기 학습**: 10-20분 (기본적인 움직임)
- **중급 학습**: 30-60분 (영역 확장 학습)
- **고급 학습**: 1-2시간 (전략적 플레이)

In [ ]:
import time
import subprocess
from datetime import datetime

print("🎮 Keep the Land! ML-Agents 학습 시작!")
print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*60)

# 학습 시작
try:
    # 실제 학습 명령어 실행
    process = subprocess.run([
        "mlagents-learn",
        "paperio_config.yaml",
        f"--run-id={RUN_ID}",
        f"--env={ENV_NAME}",
        "--no-graphics",
        "--force"
    ], 
    cwd="/content/Project-AI-Paper.io-ML-Agents",
    capture_output=False,
    text=True)
    
    print(f"\n🎉 학습 완료! (종료 코드: {process.returncode})")
    print(f"⏰ 종료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
except KeyboardInterrupt:
    print("\n⏹️ 사용자에 의해 학습이 중단되었습니다.")
except Exception as e:
    print(f"\n❌ 오류 발생: {str(e)}")
    print("\n🔧 문제 해결 방법:")
    print("1. Unity 빌드 파일 경로 확인")
    print("2. 빌드 파일 실행 권한 확인")
    print("3. GPU 메모리 확인")
    
print("\n📁 학습 결과 폴더:")
!ls -la results/

## 📈 Step 8: 학습 결과 확인

In [ ]:
import os
import glob

print("📊 Keep the Land! AI 학습 결과 분석")
print("="*50)

# 결과 폴더 확인
results_path = f"results/{RUN_ID}"

if os.path.exists(results_path):
    print(f"✅ 학습 결과 폴더 발견: {results_path}")
    
    # ONNX 모델 파일 확인
    onnx_files = glob.glob(f"{results_path}/*/*.onnx")
    if onnx_files:
        print(f"\n🧠 학습된 AI 모델: {len(onnx_files)}개")
        for onnx_file in onnx_files:
            file_size = os.path.getsize(onnx_file) / 1024  # KB
            print(f"  📄 {onnx_file} ({file_size:.1f}KB)")
    
    # 체크포인트 확인
    checkpoint_files = glob.glob(f"{results_path}/*/*.pt")
    if checkpoint_files:
        print(f"\n💾 체크포인트: {len(checkpoint_files)}개")
        
    # 로그 파일 확인
    print("\n📝 학습 폴더 구조:")
    !find $results_path -type f | head -20
    
else:
    print("❌ 학습 결과 폴더를 찾을 수 없습니다.")
    print("전체 results 폴더 확인:")
    !ls -la results/

## 💾 Step 9: 학습된 AI 모델 다운로드

In [ ]:
from google.colab import files
import shutil
import zipfile
import glob
import os

print("💾 Keep the Land! AI 모델 다운로드 준비")
print("="*40)

# ONNX 모델 파일 찾기
onnx_files = glob.glob(f"results/{RUN_ID}/*/*.onnx")

if onnx_files:
    print(f"✅ {len(onnx_files)}개의 AI 모델 발견!")
    
    # 다운로드 폴더 생성
    download_folder = "keep_the_land_ai_models"
    os.makedirs(download_folder, exist_ok=True)
    
    # 모델 파일들을 다운로드 폴더로 복사
    for i, onnx_file in enumerate(onnx_files):
        model_name = f"KeepTheLand_AI_Model_{i+1}.onnx"
        shutil.copy2(onnx_file, f"{download_folder}/{model_name}")
        print(f"📄 {model_name} 준비 완료")
    
    # 설정 파일도 포함
    shutil.copy2("paperio_config.yaml", f"{download_folder}/training_config.yaml")
    
    # README 파일 생성
    readme_content = f"""# Keep the Land! ML-Agents 학습 결과

## 📁 파일 설명
- `KeepTheLand_AI_Model_*.onnx`: 학습된 AI 모델 파일
- `training_config.yaml`: 학습에 사용된 설정

## 🎮 Unity에서 사용 방법
1. Unity 프로젝트에서 MyAgent GameObject 선택
2. Behavior Parameters 컴포넌트의 Model 필드에 .onnx 파일 드래그
3. Behavior Type을 'Default'로 설정
4. Play 버튼 클릭하여 AI 플레이 확인

## 📊 학습 정보
- 학습 ID: {RUN_ID}
- 최대 스텝: {MAX_STEPS:,}
- 학습 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- 플랫폼: Google Colab (GPU)

## 🏆 Keep the Land! 프로젝트
Unity ML-Agents를 활용한 영역 확보 게임 AI 개발
"""
    
    with open(f"{download_folder}/README.md", "w", encoding="utf-8") as f:
        f.write(readme_content)
    
    # ZIP 파일로 압축
    zip_filename = f"KeepTheLand_AI_Models_{RUN_ID}.zip"
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(download_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, download_folder)
                zipf.write(file_path, arcname)
    
    print(f"\n📦 {zip_filename} 생성 완료!")
    print(f"📊 파일 크기: {os.path.getsize(zip_filename) / 1024:.1f}KB")
    
    # 다운로드 시작
    print("\n⬇️ 다운로드 시작...")
    files.download(zip_filename)
    
    print("\n🎉 다운로드 완료!")
    print("Unity 프로젝트에서 .onnx 파일을 사용하여 AI를 테스트해보세요!")
    
else:
    print("❌ ONNX 모델 파일을 찾을 수 없습니다.")
    print("학습이 완료되었는지 확인해주세요.")

## 🎯 학습 완료 요약

### ✅ 완료된 작업
1. **환경 설정**: Python 3 + ML-Agents 설치
2. **프로젝트 준비**: Keep the Land! ML-Agents 프로젝트 클론
3. **Unity 빌드**: 헤드리스 빌드 업로드 및 권한 설정
4. **학습 설정**: PPO 알고리즘 + 최적화된 하이퍼파라미터
5. **AI 학습**: 4개 에이전트의 강화학습 진행
6. **모델 저장**: ONNX 형식으로 학습된 AI 모델 생성

### 🧠 AI가 학습한 전략들
- **영역 확장**: 효율적인 경로로 최대 영역 확보
- **생존 전략**: 위험 상황에서의 안전한 복귀
- **공격 전략**: 상대방 궤적을 끊어 점수 획득
- **위험 회피**: 벽 충돌이나 자기 궤적 충돌 방지

### 🚀 다음 단계
1. **Unity 테스트**: 다운로드한 .onnx 모델을 Unity에서 테스트
2. **성능 평가**: AI의 플레이 스타일과 승률 분석
3. **하이퍼파라미터 조정**: 더 나은 성능을 위한 재학습
4. **다양한 시나리오**: 다른 맵이나 규칙으로 추가 학습

---

### 🏆 Keep the Land! 프로젝트
**영역 확보 게임 AI 개발**: Unity + ML-Agents + PPO 강화학습을 활용한 지능형 게임 AI 개발

**🎮 프로젝트 특징**:
- 실시간 4인 멀티플레이어 AI
- 8차원 관찰 공간으로 효율적인 상황 인식
- 영역 확장, 생존, 공격의 균형잡힌 전략
- Google Colab GPU를 활용한 클라우드 학습

**💡 기술 스택**: Unity 2022.3 LTS, ML-Agents 2.0, PPO Algorithm, Python 3.8+, TensorBoard